In [4]:
import sys

sys.path.append("../src")

# Quickstart 4: Custom encoder and decoder architectures

So far, we've used a simple integral operator (inspired by Deep Sets) as the encoder and a coordinate MLP decoder (inspired by neural radiance fields/NeRFs), as described in section 2.5 of [the paper](https://arxiv.org/pdf/2408.01362).
These architectures perform well for the scientific data we've considered in the previous notebooks, and have relatively low parameter counts.
You might like to try other encoder/decoder architectures to compare and further enhance the performance of your FVAE or FAE.

### Built-in encoders

**Mesh-invariant**.
- `functional_autoencoders.encoders.fno_encoder.FNOEncoder`: an encoder based on the Fourier neural operator (FNO; [Li et al., 2021](https://arxiv.org/pdf/2010.08895)) followed by a learnable function-to-vector mapping.
- `functional_autoencoders.encoders.lno_encoder.LNOEncoder`: an encoder based on the low-rank neural operator (LNO; [Kovachki et al., 2023](https://www.jmlr.org/papers/volume24/21-1524/21-1524.pdf)) followed by a learnable function-to-vector mapping.
- `functional_autoencoders.encoders.pooling_encoder.PoolingEncoder`: the familiar Deep Set-style encoder discussed in quickstart notebooks 1 and 2.

**Fixed-resolution**.
- `functional_autoencoders.encoders.cnn_encoder.CNNEncoder`: a CNN applied to fixed-resolution input followed by a flattening operation.
- `functional_autoencoders.encoders.mlp_encoder.MLPEncoder`: a standard MLP applied to fixed-resolution input, similar to what is done in the variational autoencoding neural operator (VANO; Seidman et al., 2023).

### Built-in decoders

**Mesh-invariant**.
- `functional_autoencoders.decoders.fno_decoder.FNODecoder`: a decoder based on the FNO. First, the latent vector $z \in \mathbb{R}^{d_{\mathcal{Z}}}$ is lifted to a function $u \colon [0, 1]^{d} \to \mathbb{R}^{d_{\mathcal{Z}} + m}$ that takes the constant value $z_{i}$, $i = 1, \dots, d_{\mathcal{Z}}$ in the first $d_{\mathcal{Z}}$ components and is the identity function in the remaining components. Then an FNO is applied to this lifted function.
- `functional_autoencoders.decoders.linear_decoder.LinearDecoder`: A [DeepONet](https://arxiv.org/abs/1910.03193)-style decoder based on a linear combination of learned basis functions.
- `functional_autoencoders.decoders.nonlinear_decoder.NonlinearDecoder`: The familiar coordinate MLP-style decoder discussed in quickstart notebooks 1 and 2.

**Fixed-resolution**.
- `functional_autoencoders.decoders.cnn_decoder.CNNDecoder`: a decoder based on CNNs with transposed convolutions to map from the latent vector to a fixed-resolution output.

You can try any of these encoders or decoders in [`quickstart/1_FVAE.ipynb`](./1_FVAE.ipynb) and [`quickstart/2_FAE.ipynb`](./2_FAE.ipynb) by swapping out the lines


In [ ]:
encoder = ...
decoder = ...

If none of these suit your dataset, it's easy to roll out your own encoder and decoder architectures by extending the `Encoder` and `Decoder` base classes.
Both of these are derived from the Flax [`nn.Module`](???) class, and work in much the same way as `nn.Module`.
There are [two ways of defining a Flax `nn.Module`](https://flax.readthedocs.io/en/latest/guides/flax_fundamentals/setup_or_nncompact.html): 

1. Using `@nn.compact`, as we'll show below; or
2. Using a more explicit PyTorch-style `setup` method.

We'll focus on using `@nn.compact` in the examples below, but you can read the Flax documentation to find out more about the `setup` method if you prefer.


### A custom encoder

Let's set up a trivial encoder

$$\tilde{f}(u) = (1, 1, \dots, 1)^{T} \int_{\Omega} u(x) \,\mathrm{d} x,$$

which just returns a constant vector where each component is the spatial mean of the input function $u$.
The encoder is responsible for taking care of the latent dimension `latent_dim`, and should also take a boolean parameter `is_variational` which determines whether the architecture is for FVAE or FAE. 
In practice, for FVAE the encoder must output a vector of twice the latent dimension to accommodate both the mean vector and the diagonal entries of the covariance matrix.

In [5]:
import flax.linen as nn
import jax.numpy as jnp
from functional_autoencoders.encoders import Encoder


class MeanEncoder(Encoder):
    latent_dim: int = 64

    @nn.compact
    def __call__(self, u, x, train=False):
        u = jnp.reshape(u, (u.shape[0], -1))
        d_out = self.latent_dim * 2 if self.is_variational else self.latent_dim
        u = jnp.mean(u, axis=1, keepdims=True) * jnp.ones((u.shape[0], d_out))
        u = nn.Dense(u.shape[-1], use_bias=False)
        return u

### A custom decoder

In a similar fashion, let's set up a trivial decoder 

$$ g(z)(x) = Az + b,$$

where $A \in \mathbb{R}^{m \times d_{\mathcal{Z}}}$ and $b \in \mathbb{R}^{m}$, which returns a constant function $u \colon \Omega \subset \mathbb{R}^{d} \to \mathbb{R}^{m}$ taking value $Az + b \in \mathbb{R}^{m}$ across the whole domain.

In [ ]:
from functional_autoencoders.decoders import Decoder


class ConstantFunctionDecoder(Decoder):
    out_dim: int

    @nn.compact
    def __call__(self, z, x, train=False):
        z = nn.Dense(self.out_dim)(z)
        print(z.shape)
        z = jnp.reshape(z, (-1, 1, self.out_dim))
        z = jnp.repeat(z, x.shape[1], axis=0)
        return z

## What next?

You've reached the end of the quickstart series for FVAE/FAE. 

If you're looking for more, why not raise an issue on GitHub or submit a pull request? We're eager to see applications and extensions of the FVAE/FAE methodology and hope this series of notebooks has been useful!